In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
%matplotlib inline

In [2]:
sealion_types = ["adult_males", 
    "subadult_males",
    "adult_females",
    "juveniles",
    "pups"]

In [3]:
sealion_sizes_px = {
    "adult_males": 72,
    "subadult_males": 64,
    "adult_females": 56,
    "juveniles": 32,
    "pups": 20
}

In [4]:
def gaussian2d(x, y, sx=1, sy=1, x0=0, y0=0, A=1):
    sxx = 2 * sx * sx
    syy = 2 * sy * sy
    xc = x - x0
    yc = y - y0
    xcc = xc * xc
    ycc = yc * yc
    return A * np.exp(-(xcc/sxx + ycc/syy))

In [15]:
def mask_gaussian(size):
    sigma = size / 3.0  # 97% of the gaussian in the size * size square
    half = int((sigma * 5 ) * 2)  # to be sure that the limit of the mask have a value close to 0 (avoid artifact on density map)
    im = np.zeros((2 * half + 1, 2 * half + 1), np.float32)
    for y in range(-half, half + 1):
        for x in range(-half, half + 1):
            im[y + half, x + half] = gaussian2d(x, y, sigma, sigma, 0, 0, 1)
    total_sum = np.sum(im)
    im /= total_sum
    return im

In [69]:
sealion_gaussian = {}
for sealion, size in sealion_sizes_px.items():
    sealion_gaussian[sealion] = mask_gaussian(size)
    print(sealion, np.max(sealion_gaussian[sealion]))

adult_males 0.000276311
juveniles 0.00139882
pups 0.00358099
subadult_males 0.000349706
adult_females 0.000456758


In [96]:
sealion_position_prob = {}
patch_size = 224
for sealion, size in sealion_sizes_px.items():
    kernel_size = patch_size - size
    if kernel_size % 2 == 0:
        kernel_size += 1
    sealion_position_prob[sealion] = np.ones((kernel_size, kernel_size))

In [91]:
def create_density_map(shape, dots, mask, prob=1.0):
    im = np.zeros(shape, np.float32)
    
    # Reweight mask so that the sum is equal to prob
    total_mask_weight = np.sum(mask)
    mask_weighted = (prob/total_mask_weight) * mask 
    
    hmask, wmask = mask.shape
    hmask2 = hmask//2
    wmask2 = wmask//2
    
    for (xf, yf) in dots:
        x = int(round(xf))
        y = int(round(yf))
        xmin = max(0, x - wmask2)
        xmax = min(im.shape[1] - 1, x + wmask2)
        ymin = max(0, y - hmask2)
        ymax = min(im.shape[0] - 1, y + hmask2)
       
        dy_top = y - ymin
        dy_bottom = ymax - y
        dx_left = x - xmin
        dx_right = xmax - x
        im[ymin:ymax+1, xmin:xmax+1] += mask_weighted[hmask2 - dy_top: hmask2 + dy_bottom + 1,\
                                                      wmask2 - dx_left: wmask2 + dx_right + 1] 
    return im

In [119]:
def create_position_probability_map(shape, dots, sealion_probabilities, background_weight = 2, patch_size=(224, 224)):
    im = np.zeros(shape, np.float32)
    for s, p in sealion_probabilities.items():
        if s in dots:
            im += create_density_map(shape, dots[s], sealion_position_prob[s], p)
    p_sealion = np.sum(im)
    n_empty = np.count_nonzero(im == 0)
    n_border = patch_size[1] * im.shape[0] + (im.shape[1] - patch_size[1]) * patch_size[0]
    proba_empty = p_sealion/(n_empty - n_border) * background_weight
    im[im==0] = proba_empty
    im[:,:patch_size[1]//2] = 0
    im[:patch_size[0]//2,:] = 0
    im[:,-patch_size[1]//2:] = 0
    im[-patch_size[0]//2:,:] = 0
    return im/np.sum(im)  #normalize

In [114]:
def show_density(im, density):
    max_density = np.max(density)
    normalized_density = density/max_density
    im_fp32 = im.astype(np.float32)
    im_masked = im_fp32.copy()
    im_masked[:,:,0] = (im_fp32[:,:,0] * normalized_density)
    im_masked[:,:,1] = (im_fp32[:,:,1] * normalized_density)
    im_masked[:,:,2] = (im_fp32[:,:,2] * normalized_density)
    return im_masked.astype(np.uint8), (normalized_density * 255).astype(np.uint8)

In [115]:
with open("../data/sealion/TrainDots/872.pkl", "rb") as ifile:
    dots = pickle.load(ifile)

In [94]:
im_train = cv2.imread("../data/sealion/Train/872.jpg")

In [109]:
sealion_probabilities = {'adult_females': 0.048142699511667254,
 'adult_males': 0.3351506883474506,
 'juveniles': 0.089826648353188876,
 'pups': 0.11096914409391793,
 'subadult_males': 0.41591081969377525}

In [122]:
location_prob = create_position_probability_map(im_train.shape[:-1], dots, sealion_probabilities)

In [71]:
density_female = create_density_map(im_train.shape[:-1], dots['adult_females'], sealion_gaussian['adult_females'])

In [123]:
def sample_patchs(im_train, location_prob, n_patches):
    h, w = location_prob.shape
    random_indices = np.random.choice(h * w, size=(n_patches, ), replace=False, p=location_prob.ravel())
    points = []
    for index in random_indices:
        x = index%w
        y = index//w
        points.append((x, y))
    for p in points:
        dst = cv2.circle(im_train, p, 10, (0, 0, 255), -1)
    return dst

In [126]:
dst = sample_patchs(im_train, location_prob, 1000)

In [127]:
cv2.imwrite("../data/sample.png", dst)

True

In [128]:
im_density, mask_density = show_density(im_train, location_prob)

In [118]:
cv2.imwrite("../data/imk.png", im_density)
cv2.imwrite("../data/kk.png", mask_density)

True

In [24]:
np.sum(density_female)

131.99989